In [80]:
# imports
import pandas as pd
import numpy as np
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [81]:
# get data and create dum
cancer_df =pd.read_csv(r'..\Cases\Cancer\Cancer.csv',index_col=0)
cancer_df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
subjid,,,,,,,,,,
1,40-49,premeno,15 to 19,0 to 2,yes,three,right,left_up,no,recurrence-events
2,50-59,ge40,15 to 19,0 to 2,no,one,right,central,no,no-recurrence-events
3,50-59,ge40,35 to 39,0 to 2,no,two,left,left_low,no,recurrence-events
4,40-49,premeno,35 to 39,0 to 2,yes,three,right,left_low,yes,no-recurrence-events
5,40-49,premeno,30 to 34,3 to 5,yes,two,left,right_up,no,recurrence-events


In [82]:
y = cancer_df['Class']
X = cancer_df.drop('Class',axis=1)
X.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
subjid,,,,,,,,,
1,40-49,premeno,15 to 19,0 to 2,yes,three,right,left_up,no
2,50-59,ge40,15 to 19,0 to 2,no,one,right,central,no
3,50-59,ge40,35 to 39,0 to 2,no,two,left,left_low,no
4,40-49,premeno,35 to 39,0 to 2,yes,three,right,left_low,yes
5,40-49,premeno,30 to 34,3 to 5,yes,two,left,right_up,no


In [83]:
le=LabelEncoder()
y=le.fit_transform(y)

In [84]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25,stratify=y)

In [85]:
ohe=OneHotEncoder(sparse_output=False,drop='first')
x_train_encoded=ohe.fit_transform(X_train)
x_test_encoded=ohe.transform(X_test)
x_train_encoded.shape

(200, 34)

In [86]:
nb = BernoulliNB()
nb.fit(x_train_encoded,y_train) # calculates all aporiat probalblities

BernoulliNB()

In [87]:
probas = nb.predict_proba(x_test_encoded) # calculates posterior proabalities
roc_auc_score(y_test,probas[:,1])

0.7653846153846154

In [91]:
import numpy as np
alpha=np.linspace(0,5,20)
score=[]
for i in alpha:
    nb = BernoulliNB(alpha=i)
    nb.fit(x_train_encoded,y_train)
    probas = nb.predict_proba(x_test_encoded) # calculates posterior proabalities
    auc_score=roc_auc_score(y_test,probas[:,1])
    score.append([i,auc_score])
df_scores=pd.DataFrame(score,columns=['Alpha','Auc score'])
df_scores.sort_values(by='Auc score',ascending=False,inplace=True)
df_scores.head()

,Alpha,Auc score
3,0.789474,0.767308
2,0.526316,0.766667
1,0.263158,0.766667
0,0.000000,0.766026
9,2.368421,0.765385
